In [10]:
import helpers
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
from torch.optim import lr_scheduler
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, classification_report

In [2]:
exercise_mapping = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat': 5,
    'Stretch': 6
}
set_mapping = {
    'Correct' : 0,
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6
}

In [3]:
#Importing pre dataset
train_df = pd.read_parquet('train_set_augmented.parquet', engine='pyarrow')
test_df = pd.read_parquet('test_set_augmented.parquet', engine='pyarrow')

In [4]:
video_indices = test_df.groupby('video_id').size().values
video_indices = np.insert(video_indices, 0, 0)
video_indices = np.cumsum(video_indices)

In [5]:
get_mlp3x256 = lambda: torch.nn.Sequential(

    torch.nn.Flatten(),
    torch.nn.Linear(100, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

In [6]:
get_physio_mlp = lambda: torch.nn.Sequential(

    nn.Flatten(),
    nn.Linear(107, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Linear(512, 7)
)

# Training the model to predict exercises
We first split the train and test set


In [8]:
X_train, X_test, y_train, y_test = helpers.import_data_exercise(train_df, test_df, exercise_mapping, True)

We then define everything we need to train our model

In [13]:
#Defines hyperparameters
learning_rate = 2e-3
batch_size = 128

# Creates an instance of the dataset
dataset = TensorDataset(X_train, y_train)

# Creates a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Defines the model
model = get_mlp3x256()
model.train()

# Defines the path to save the model's state dictionnary
model_path = 'MLPEX.path'

# Defines loss function
criterion = nn.CrossEntropyLoss()

# Creates an optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Creates a scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)

We procede to train the model. It converges to sufficiently trained state to perform 100% accuracy on test set in a single iteration. Note that the already trained model state dictionnary can be loaded from the path 'MLPEX_trained.path' by uncommenting the cell below.

In [12]:
#model.load_state_dict(torch.load('MLPEX_trained.path'))
#helpers.test_accuracy(model, X_test,y_test,video_indices)

Accuracy on each frame: 0.9606
Accuracy for videos: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        60
           2       1.00      1.00      1.00        60
           3       1.00      1.00      1.00        60
           4       1.00      1.00      1.00        60
           5       1.00      1.00      1.00        60
           6       1.00      1.00      1.00        60

    accuracy                           1.00       420
   macro avg       1.00      1.00      1.00       420
weighted avg       1.00      1.00      1.00       420



In [15]:
# Trains the model
helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)

# Prints a report on the accuracy and F1 score
helpers.test_accuracy(model, X_test,y_test,video_indices)

Accuracy on each frame: 0.9605
Accuracy for videos: 0.9952
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00        60
           2       1.00      1.00      1.00        60
           3       1.00      1.00      1.00        60
           4       0.97      1.00      0.98        60
           5       1.00      0.97      0.98        60
           6       1.00      1.00      1.00        60

    accuracy                           1.00       420
   macro avg       1.00      1.00      1.00       420
weighted avg       1.00      1.00      1.00       420



# Training of MLP 3x256 to predict sets

We first split the train and test set, this time we one-hot encoded our predictions of the exercise in the dataset

In [7]:
X_train, X_test, y_train, y_test = helpers.import_data_set(train_df, test_df, set_mapping, True)

In [11]:
#Defines hyperparameters
learning_rate = 0.001
batch_size = 128

# Creates an instance of the dataset
dataset = TensorDataset(X_train, y_train)

# Creates a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Defines the model
model = get_physio_mlp()
model.train()

# Defines the path to save the model's state dictionnary
model_path = 'PhysioMLP.path'

# Defines loss function
criterion = nn.CrossEntropyLoss()

# Creates an optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Creates a scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

In [14]:
model.load_state_dict(torch.load('PhysioMLP2.path'))
helpers.test_accuracy(model, X_test,y_test,video_indices)

Accuracy on each frame: 0.4033
Accuracy for videos: 0.5905
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.67      0.62       140
           1       0.64      0.58      0.61        76
           2       0.62      0.58      0.60        72
           3       0.60      0.52      0.55        64
           4       0.62      0.53      0.57        40
           5       0.45      0.56      0.50        16
           6       0.42      0.42      0.42        12

    accuracy                           0.59       420
   macro avg       0.56      0.55      0.55       420
weighted avg       0.59      0.59      0.59       420



In [ ]:
for i in range(30):
  # Trains the model
  helpers.train_model(1, model, optimizer, criterion, dataloader, scheduler, model_path)
  # Prints a report on the accuracy and F1 score
  helpers.test_accuracy(model, X_test,y_test,video_indices)

Processing: 31.67%